In [ ]:



# %% — Cell 1: Install Dependencies
# ──────────────────────────────────────────────────────────────────

import subprocess, sys

def pip_install(*packages):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                           *packages])

print("📦 [1/5] Core packages...")
pip_install("Pillow>=9.1.0", "packaging", "numpy<2.0",
            "filelock>=3.9.0", "huggingface_hub>=0.25.0")

print("📦 [2/5] PyTorch...")
pip_install("torch", "torchvision", "torchaudio", "scipy>=1.11.0")

print("📦 [3/5] HuggingFace, audio, evaluation...")
pip_install("transformers>=4.46.0", "accelerate", "datasets[audio]>=3.0.0,<4.0.0",
            "evaluate", "jiwer", "soundfile", "librosa", "requests")

print("📦 [4/5] LoRA + quantization (PEFT + bitsandbytes)...")
pip_install("peft>=0.7.0", "bitsandbytes>=0.41.0")

print("📦 [5/5] Mozilla Data Collective SDK...")
pip_install("datacollective", "requests")

print("\n✅ All packages installed.")
print("⚠️  IMPORTANT: RESTART your runtime now (Runtime → Restart runtime).")
print("   Then skip this cell and continue from Cell 2.")


# %% — Cell 2: Authenticate & Verify
# ──────────────────────────────────────────────────────────────────

import os
from huggingface_hub import login

# ═══════════════════════════════════════════════════════════════════
# ⚠️  SECURITY: Set these as environment variables instead of
#     hardcoding. Rotate keys if you've ever shared them publicly.
#
#     export HF_TOKEN="hf_..."
#     export MDC_API_KEY="752210..."
#     export MDC_CLIENT_ID="mdc_4c..."
# ═══════════════════════════════════════════════════════════════════

HF_TOKEN = os.environ.get("HF_TOKEN", "your_key")
MDC_API_KEY = os.environ.get("MDC_API_KEY", "your_key")
MDC_CLIENT_ID = os.environ.get("MDC_CLIENT_ID", "your_ID")

os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["MDC_API_KEY"] = MDC_API_KEY
os.environ["MDC_API_URL"] = "https://datacollective.mozillafoundation.org/api"
os.environ["MDC_DOWNLOAD_PATH"] = os.path.expanduser("~/.mozdata/datasets")

login(token=HF_TOKEN, add_to_git_credential=False)

import torch
print(f"✅ Authenticated (HF + MDC). torch={torch.__version__}")
print(f"   CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


# %% — Cell 3: Configuration
# ──────────────────────────────────────────────────────────────────

import multiprocessing

# ── Model ──
MODEL_ID = "openai/whisper-large-v3"      # 1.5B params, SOTA zero-shot ASR
USE_LORA = True                            # Required for large-v3 on ≤24GB VRAM
LORA_R = 32                                # LoRA rank (32 = good balance)
LORA_ALPHA = 64                            # LoRA alpha (2x rank is standard)
LOAD_IN_4BIT = True                        # 4-bit quantization to fit in VRAM

# ── Data ──
SAMPLING_RATE = 16000
MAX_AUDIO_LENGTH_S = 30                    # Whisper's max input length
MAX_LABEL_LENGTH = 448                     # Max token length for labels
USE_ENGLISH_DIALECTS = True                # +31h British Isles accents
USE_COMMON_VOICE = True                    # +accent-filtered Common Voice via MDC
CV_ACCENT_HOURS_CAP = 40                   # Cap CV data at ~40h to balance

# Mozilla Data Collective dataset ID (from the URL you shared)
MDC_DATASET_ID = "cmko7havo02f5nw07rbwwhowe"

# ── Training ──
NUM_EPOCHS = 15                            # More data → fewer epochs needed
BATCH_SIZE = 4                            # Per-device (large-v3 is memory hungry)
GRAD_ACCUM = 8                            # Effective batch = 2 * 16 = 32
LEARNING_RATE = 1e-4                       # Higher LR is fine with LoRA
WARMUP_STEPS = 500
EVAL_STEPS = 500                           # Evaluate every 500 steps
SAVE_TOTAL_LIMIT = 3

# ── Device ──
device = "cuda" if torch.cuda.is_available() else "cpu"
NUM_WORKERS = min(4, multiprocessing.cpu_count())

print("=" * 60)
print("CONFIGURATION")
print("=" * 60)
print(f"  Model:       {MODEL_ID}")
print(f"  LoRA:        r={LORA_R}, alpha={LORA_ALPHA}, 4-bit={LOAD_IN_4BIT}")
print(f"  Epochs:      {NUM_EPOCHS}")
print(f"  Effective BS: {BATCH_SIZE * GRAD_ACCUM}")
print(f"  LR:          {LEARNING_RATE}")
print(f"  Device:      {device}")
print(f"  MDC Dataset: {MDC_DATASET_ID}")
print("=" * 60)


# %% — Cell 4: Load Processor
# ──────────────────────────────────────────────────────────────────

from transformers import WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer

print("📦 Loading Whisper large-v3 processor...")

feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_ID)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_ID, language="english", task="transcribe")
processor = WhisperProcessor.from_pretrained(MODEL_ID, language="english", task="transcribe")

print(f"✅ Processor loaded. Vocab size: {tokenizer.vocab_size}")


# %% — Cell 5: Load & Combine Accent Datasets
# ──────────────────────────────────────────────────────────────────
#
# This is the MAIN DATA CELL — combines:
#   1. EdAcc (primary benchmark target)
#   2. english_dialects (British Isles regional accents)
#   3. Common Voice v24 en-AU subset via Mozilla Data Collective
#
# The key fix: Common Voice is no longer on HuggingFace (moved Oct 2025).
# We now download via the MDC Python SDK and load the CSV + MP3 files.
#
# DTYPE FIX: All audio arrays are normalized to np.float32 before being
# added to sample lists, preventing ArrowInvalid errors when combining.
#

from datasets import load_dataset, Audio, Dataset
from collections import Counter
import csv
import glob
import os
import tarfile
import soundfile as sf
import librosa
import numpy as np

print("=" * 60)
print("LOADING ACCENT DATASETS")
print("=" * 60)

all_train_samples = []
all_eval_samples = []


# ══════════════════════════════════════════════════════════════════
# FIX: Helper to normalize audio dicts for Arrow compatibility
# ══════════════════════════════════════════════════════════════════
def normalize_audio_to_float32(audio):
    """
    Normalize an audio dict to a CONSISTENT structure with float32 arrays.
    
    This fixes TWO issues that cause ArrowInvalid errors:
      1. dtype mismatch: HF datasets return float64, librosa returns float32
      2. key mismatch: HF audio dicts have {"path","array","sampling_rate"}
         but manually-built dicts only have {"array","sampling_rate"}.
         Arrow cannot reconcile dicts with different key sets in one column.
    
    Output is ALWAYS: {"array": np.float32, "sampling_rate": int}
    """
    if isinstance(audio, dict):
        arr = audio.get("array")
        sr = audio.get("sampling_rate", SAMPLING_RATE)
        if arr is not None and hasattr(arr, "astype"):
            arr = arr.astype(np.float32)
        # Return ONLY the two required keys — strip "path", "bytes", etc.
        return {"array": arr, "sampling_rate": int(sr)}
    return audio


# ── Helper: audio handling for both old and new datasets lib ──────
def get_audio_array_and_sr(audio):
    """Extract audio array and sample rate from either dict or torchcodec AudioDecoder.
    
    IMPORTANT: After Arrow round-trip (save to disk then read back),
    numpy arrays get deserialized as plain Python lists. We must
    always ensure the output is a np.float32 ndarray.
    """
    try:
        if isinstance(audio, dict):
            arr = audio["array"]
            # Always convert to numpy float32 — arr may be a list after Arrow round-trip
            arr = np.asarray(arr, dtype=np.float32)
            return arr, audio["sampling_rate"]
        arr = audio["array"]
        arr = np.asarray(arr, dtype=np.float32)
        return arr, audio["sampling_rate"]
    except (TypeError, KeyError):
        try:
            samples = audio.get_all_samples()
            return samples.data.numpy().squeeze().astype(np.float32), samples.sample_rate
        except Exception:
            return None, None


def is_valid_sample(sample):
    text = sample.get("text", "")
    if not text or len(text.strip()) < 2:
        return False
    audio = sample.get("audio")
    if audio is not None:
        try:
            arr, sr = get_audio_array_and_sr(audio)
            if arr is not None:
                duration = len(arr) / (sr or SAMPLING_RATE)
                if duration < 0.5 or duration > MAX_AUDIO_LENGTH_S:
                    return False
        except Exception:
            pass
    return True


# ══════════════════════════════════════════════════════════════════
# Dataset 1: EdAcc (primary — this is our target benchmark)
# ══════════════════════════════════════════════════════════════════
print("\n📥 [1/3] Loading EdAcc (40h, diverse international accents)...")
edacc = load_dataset("edinburghcstr/edacc", token=HF_TOKEN)

edacc_val = edacc["validation"]
edacc_test = edacc["test"]

edacc_val_filtered = edacc_val.filter(is_valid_sample)
edacc_test_filtered = edacc_test.filter(is_valid_sample)

# Use 90% of validation for training, 10% for eval during training
edacc_split = edacc_val_filtered.train_test_split(test_size=0.1, seed=42)
edacc_train_portion = edacc_split["train"]
edacc_eval_portion = edacc_split["test"]

print(f"   EdAcc train: {len(edacc_train_portion)} samples")
print(f"   EdAcc eval:  {len(edacc_eval_portion)} samples")
print(f"   EdAcc test (leaderboard): {len(edacc_test_filtered)} samples")

# FIX: normalize_audio_to_float32 on every sample before appending
for sample in edacc_train_portion:
    all_train_samples.append({
        "audio": normalize_audio_to_float32(sample["audio"]),
        "text": sample["text"].strip().upper(),
        "source": "edacc"
    })
for sample in edacc_eval_portion:
    all_eval_samples.append({
        "audio": normalize_audio_to_float32(sample["audio"]),
        "text": sample["text"].strip().upper(),
        "source": "edacc"
    })


# ══════════════════════════════════════════════════════════════════
# Dataset 2: English Dialects (British Isles, 31h)
# ══════════════════════════════════════════════════════════════════
if USE_ENGLISH_DIALECTS:
    print("\n📥 [2/3] Loading english_dialects (31h, British Isles accents)...")
    DIALECT_CONFIGS = [
        'irish_male', 'midlands_female', 'midlands_male',
        'northern_female', 'northern_male', 'scottish_female',
        'scottish_male', 'southern_female', 'southern_male',
        'welsh_female', 'welsh_male'
    ]
    dialect_samples = []
    for config_name in DIALECT_CONFIGS:
        try:
            ds = load_dataset("ylacombe/english_dialects", config_name,
                              split="train", streaming=True)
            count = 0
            for sample in ds:
                text = sample.get("text", "")
                if text and len(text.strip()) >= 2:
                    # FIX: normalize audio to float32
                    dialect_samples.append({
                        "audio": normalize_audio_to_float32(sample["audio"]),
                        "text": text.strip().upper(),
                        "source": "english_dialects"
                    })
                    count += 1
            print(f"   {config_name}: {count} samples")
        except Exception as e:
            print(f"   ⚠️  {config_name} failed: {e}")

    if dialect_samples:
        n_eval = max(1, len(dialect_samples) // 20)
        all_eval_samples.extend(dialect_samples[:n_eval])
        all_train_samples.extend(dialect_samples[n_eval:])
        print(f"   english_dialects total: {len(dialect_samples) - n_eval} train, {n_eval} eval")
    else:
        print("   ⚠️  No dialect samples loaded")


# ══════════════════════════════════════════════════════════════════
# Dataset 3: Common Voice v24 English (en-AU subset) via MDC
# ══════════════════════════════════════════════════════════════════
#
# This dataset is the "Common Voice v24 English - en-AU subset for
# Everything Open 2026" from Mozilla Data Collective.
#
# Structure (different from standard Common Voice!):
#   Archive:  common-voice-v24-english-en-au-subset-fo-0447e8a6.tar.gz
#   Metadata: commonvoice-v24_en-AU.csv  (CSV, NOT TSV)
#   Audio:    audios/<id>.mp3             (NOT clips/)
#   Columns:  client_id, path, sentence, sentence_id, sentence_domain,
#             up_votes, down_votes, age, gender, accents, locale,
#             segment, duration_ms
#
# ALL 55,673 samples are already Australian-accented — no filtering
# needed. Accents include: "Australian English", "General Australian",
# "South Australia", "Educated Australian Accent",
# "Sydney - middle eastern seaboard Australian", "Queenslandish"
#
# CC0-1.0 licensed.
#

if USE_COMMON_VOICE:
    print("\n📥 [3/3] Loading Common Voice v24 en-AU via Mozilla Data Collective...")
    print(f"   Dataset ID: {MDC_DATASET_ID}")
    print(f"   This is a pre-filtered Australian accent subset (~55K samples, 1.92GB)")

    cv_samples = []
    cv_loaded = False

    # ── Step 1: Download the dataset archive ─────────────────────
    #
    # Known issue: MDC SDK v0.0.34 DataCollective.get_dataset()
    # can silently fail (prints HTTP 400 but doesn't raise).
    # The Cloudflare R2 presigned URLs sometimes reject SDK requests.
    #
    # Strategy:
    #   1. Check if file already exists (uploaded manually or prior run)
    #   2. Try SDK, then VERIFY files actually landed on disk
    #   3. Try REST API with minimal headers for R2 compatibility
    #   4. Give clear upload instructions if all else fails
    #

    try:
        MDC_DOWNLOAD_DIR = os.environ.get("MDC_DOWNLOAD_PATH",
                                           os.path.expanduser("~/.mozdata/datasets"))
        os.makedirs(MDC_DOWNLOAD_DIR, exist_ok=True)

        download_success = False
        archive_dest = None  # Will be set when we find/download the archive
        cv_extract_dir_precheck = None  # Set if we find already-extracted data

        # ── Pre-check: look for already-downloaded/uploaded files ──
        # Search common locations for the tar.gz
        SEARCH_DIRS = [
            MDC_DOWNLOAD_DIR,
            os.path.expanduser("~"),
            os.path.expanduser("~/Downloads"),
            "/content",                # Google Colab
            "/content/drive/MyDrive",  # Colab + Drive mounted
            "/tmp",
            ".",
        ]

        print("   Checking for existing dataset files...")
        for search_dir in SEARCH_DIRS:
            if not os.path.exists(search_dir):
                continue
            try:
                for f in os.listdir(search_dir):
                    fpath = os.path.join(search_dir, f)
                    # Match tar.gz archives
                    if (f.endswith(('.tar.gz', '.tgz'))
                            and ("common" in f.lower() or "en-au" in f.lower()
                                 or "commonvoice" in f.lower())):
                        fsize = os.path.getsize(fpath)
                        if fsize > 500_000_000:  # > 500MB = likely the real archive
                            archive_dest = fpath
                            print(f"   ✅ Found existing archive: {archive_dest} ({fsize / 1e9:.2f}GB)")
                            download_success = True
                            break
                    # Match already-extracted directory
                    if os.path.isdir(fpath):
                        csv_check = glob.glob(os.path.join(fpath, "*.csv"))
                        has_audios = any(os.path.isdir(os.path.join(fpath, d)) for d in ["audios", "audio_files", "clips"])
                        if csv_check and has_audios:
                            cv_extract_dir_precheck = fpath
                            print(f"   ✅ Found extracted data: {cv_extract_dir_precheck}")
                            download_success = True
                            break
                        # One level deeper
                        for sub in os.listdir(fpath) if os.path.isdir(fpath) else []:
                            sub_path = os.path.join(fpath, sub)
                            if os.path.isdir(sub_path):
                                csv_check2 = glob.glob(os.path.join(sub_path, "*.csv"))
                                has_audios2 = any(os.path.isdir(os.path.join(sub_path, d)) for d in ["audios", "audio_files", "clips"])
                                if csv_check2 and has_audios2:
                                    cv_extract_dir_precheck = sub_path
                                    print(f"   ✅ Found extracted data: {cv_extract_dir_precheck}")
                                    download_success = True
                                    break
                if download_success:
                    break
            except PermissionError:
                continue

        # ── Attempt 1: MDC Python SDK ─────────────────────────────
        if not download_success:
            try:
                import datacollective
                print(f"   [SDK] datacollective v{datacollective.__version__} installed")

                if hasattr(datacollective, 'save_dataset_to_disk'):
                    print(f"   [SDK] Trying save_dataset_to_disk()...")
                    try:
                        result = datacollective.save_dataset_to_disk(MDC_DATASET_ID)
                        print(f"   [SDK] Returned: {result}")
                    except Exception as e:
                        print(f"   [SDK] save_dataset_to_disk error: {e}")

                elif hasattr(datacollective, 'DataCollective'):
                    print(f"   [SDK] Trying DataCollective().get_dataset()...")
                    try:
                        client = datacollective.DataCollective()
                        result = client.get_dataset(MDC_DATASET_ID)
                        print(f"   [SDK] get_dataset returned (check for download errors above)")
                    except Exception as e:
                        print(f"   [SDK] get_dataset error: {e}")

                # CRITICAL: Verify files actually appeared on disk
                # The SDK can print errors but NOT raise them
                for root, dirs, files in os.walk(MDC_DOWNLOAD_DIR):
                    for f in files:
                        fpath = os.path.join(root, f)
                        if f.endswith(('.tar.gz', '.tgz', '.tar')) and os.path.getsize(fpath) > 500_000_000:
                            archive_dest = fpath
                            download_success = True
                            print(f"   ✅ SDK download verified: {fpath}")
                            break
                    if download_success:
                        break

                if not download_success:
                    print(f"   ⚠️  SDK ran but no archive found on disk (download likely failed silently)")

            except ImportError:
                print("   [SDK] Not installed, skipping")
            except Exception as e:
                print(f"   [SDK] Error: {e}")

        # ── Attempt 2: Direct REST API ────────────────────────────
        if not download_success:
            import requests

            api_base = os.environ.get(
                "MDC_API_URL",
                "https://datacollective.mozillafoundation.org/api"
            )
            api_key = os.environ.get("MDC_API_KEY", MDC_API_KEY)

            archive_filename = "common-voice-v24-english-en-au-subset.tar.gz"
            archive_dest = os.path.join(MDC_DOWNLOAD_DIR, archive_filename)

            try:
                # Step A: Get presigned download URL
                print(f"   [API] Requesting download URL...")
                headers = {
                    "Authorization": f"Bearer {api_key}",
                    "Content-Type": "application/json",
                }
                dl_resp = requests.post(
                    f"{api_base}/datasets/{MDC_DATASET_ID}/download",
                    headers=headers,
                    timeout=30,
                )

                if dl_resp.status_code == 200:
                    dl_data = dl_resp.json()
                    presigned_url = (
                        dl_data.get("downloadUrl")
                        or dl_data.get("url")
                        or dl_data.get("download_url")
                    )

                    if presigned_url:
                        print(f"   [API] Got presigned URL, downloading (~1.92GB)...")

                        # Download with MINIMAL headers for R2 compatibility
                        # Cloudflare R2 presigned URLs can reject requests
                        # with unexpected headers (User-Agent, Accept, etc.)
                        with requests.get(
                            presigned_url,
                            stream=True,
                            timeout=3600,
                            headers={},  # Empty headers — let requests use bare minimum
                        ) as r:
                            r.raise_for_status()
                            total = int(r.headers.get("content-length", 0))
                            downloaded = 0
                            with open(archive_dest, "wb") as f:
                                for chunk in r.iter_content(chunk_size=8 * 1024 * 1024):
                                    f.write(chunk)
                                    downloaded += len(chunk)
                                    if total > 0 and downloaded % (200 * 1024 * 1024) < len(chunk):
                                        pct = downloaded / total * 100
                                        print(f"   ... {downloaded / 1e9:.2f}GB / {total / 1e9:.2f}GB ({pct:.0f}%)")

                        if os.path.getsize(archive_dest) > 500_000_000:
                            print(f"   ✅ API download complete: {archive_dest}")
                            download_success = True
                        else:
                            print(f"   ⚠️  Downloaded file too small, likely corrupted")
                            os.remove(archive_dest)
                    else:
                        print(f"   ⚠️  No download URL in response: {list(dl_data.keys())}")

                elif dl_resp.status_code == 403:
                    print(f"   ⚠️  HTTP 403 — agree to dataset terms first at:")
                    print(f"   https://datacollective.mozillafoundation.org/datasets/{MDC_DATASET_ID}")
                elif dl_resp.status_code == 401:
                    print(f"   ⚠️  HTTP 401 — check MDC_API_KEY")
                else:
                    print(f"   ⚠️  API returned {dl_resp.status_code}: {dl_resp.text[:200]}")

            except requests.exceptions.HTTPError as e:
                print(f"   ⚠️  HTTP error during download: {e}")
                print(f"   The Cloudflare R2 presigned URL rejected the request.")
                print(f"   This is a known issue with MDC's storage backend.")
            except Exception as e:
                print(f"   ⚠️  REST API error: {e}")

        # ── If all automated methods failed ───────────────────────
        if not download_success:
            print("\n" + "=" * 70)
            print("   ⚠️  AUTOMATED DOWNLOAD FAILED")
            print("   The MDC storage (Cloudflare R2) is rejecting API downloads.")
            print("   This is a known issue. Please upload the file manually:")
            print("=" * 70)
            print(f"")
            print(f"   OPTION A — SCP from your laptop:")
            print(f"   scp ~/Downloads/common-voice-v24-english-en-au-subset-fo-*.tar.gz \\")
            print(f"       your-server:{MDC_DOWNLOAD_DIR}/")
            print(f"")
            print(f"   OPTION B — wget/curl on server (copy URL from browser download):")
            print(f"   cd {MDC_DOWNLOAD_DIR}")
            print(f"   wget '<paste-download-url-here>' -O cv-en-au.tar.gz")
            print(f"")
            print(f"   OPTION C — Jupyter upload:")
            print(f"   Use the Jupyter file browser to upload the .tar.gz to:")
            print(f"   {MDC_DOWNLOAD_DIR}")
            print(f"")
            print(f"   Then re-run this cell.")
            print("=" * 70)
            raise RuntimeError(
                "Automated download failed. Upload the file manually (see instructions above), then re-run."
            )

        # ── Step 2: Find and extract the archive ──────────────────
        cv_extract_dir = cv_extract_dir_precheck  # May already be set from pre-check
        archive_path = archive_dest  # From download step (may be None)

        # Look for the downloaded archive (skip if already found extracted data)
        if cv_extract_dir is None:
            for root, dirs, files in os.walk(MDC_DOWNLOAD_DIR):
                for f in files:
                    fpath = os.path.join(root, f)
                    if f.endswith(('.tar.gz', '.tgz', '.tar')) and os.path.getsize(fpath) > 500_000_000:
                        archive_path = fpath
                        break
                # Also check if already extracted (look for CSV + audios/)
                for d in dirs:
                    dpath = os.path.join(root, d)
                    for check_dir in [dpath] + [os.path.join(dpath, sub) for sub in os.listdir(dpath) if os.path.isdir(os.path.join(dpath, sub))]:
                        csv_files = glob.glob(os.path.join(check_dir, "*.csv"))
                        for _audio_name in ["audio_files", "audios", "clips"]:
                            audios_dir = os.path.join(check_dir, _audio_name)
                            if csv_files and os.path.isdir(audios_dir):
                                cv_extract_dir = check_dir
                                break
                        if cv_extract_dir:
                            break
                    if cv_extract_dir:
                        break
                if cv_extract_dir or archive_path:
                    break

        # Extract if needed
        if cv_extract_dir is None and archive_path is not None:
            print(f"   Extracting {os.path.basename(archive_path)}...")
            extract_target = os.path.join(MDC_DOWNLOAD_DIR, "cv_extracted")
            os.makedirs(extract_target, exist_ok=True)

            with tarfile.open(archive_path) as tar:
                tar.extractall(path=extract_target)

            print(f"   Extracted to {extract_target}")

            # Find the directory containing CSV + audios/
            for root, dirs, files in os.walk(extract_target):
                csv_files = glob.glob(os.path.join(root, "*.csv"))
                if csv_files and any(d in dirs for d in ["audios", "audio_files", "clips"]):
                    cv_extract_dir = root
                    break
                # Check if audios/ is a subdirectory
                audios_candidate = None
                for _aname in ["audios", "audio_files", "clips"]:
                    _ac = os.path.join(root, _aname)
                    if os.path.isdir(_ac):
                        audios_candidate = _ac
                        break
                if csv_files and audios_candidate and os.path.isdir(audios_candidate):
                    cv_extract_dir = root
                    break

        # Last resort: scan entire download dir for the expected structure
        if cv_extract_dir is None:
            for root, dirs, files in os.walk(MDC_DOWNLOAD_DIR):
                csv_files = [f for f in files if f.endswith('.csv')]
                if csv_files and (any(d in dirs for d in ["audios", "audio_files", "clips"]) or any(f.endswith('.mp3') for f in files)):
                    cv_extract_dir = root
                    break

        if cv_extract_dir is None:
            print("   ⚠️  Could not find CSV + audio directory after download/extraction.")
            print(f"   Contents of {MDC_DOWNLOAD_DIR}:")
            for root, dirs, files in os.walk(MDC_DOWNLOAD_DIR):
                level = root.replace(MDC_DOWNLOAD_DIR, '').count(os.sep)
                indent = ' ' * 2 * level
                print(f"   {indent}{os.path.basename(root)}/")
                if level < 3:
                    sub_indent = ' ' * 2 * (level + 1)
                    for f in sorted(files)[:15]:
                        print(f"   {sub_indent}{f}")
                    if len(files) > 15:
                        print(f"   {sub_indent}... and {len(files) - 15} more files")
            raise FileNotFoundError("No CSV metadata + audio directory (audios/ or audio_files/) found in downloaded dataset")

        print(f"   📂 Found CV data at: {cv_extract_dir}")

        # ── Step 3: Find the CSV metadata file ────────────────────
        # Expected: commonvoice-v24_en-AU.csv
        csv_path = None

        known_csv = os.path.join(cv_extract_dir, "commonvoice-v24_en-AU.csv")
        if os.path.exists(known_csv):
            csv_path = known_csv
        else:
            # Find any CSV, then TSV as fallback
            csv_candidates = glob.glob(os.path.join(cv_extract_dir, "*.csv"))
            tsv_candidates = glob.glob(os.path.join(cv_extract_dir, "*.tsv"))
            all_metadata = csv_candidates + tsv_candidates
            if all_metadata:
                csv_path = max(all_metadata, key=os.path.getsize)
            else:
                raise FileNotFoundError(
                    f"No CSV/TSV metadata found in {cv_extract_dir}. "
                    f"Files: {os.listdir(cv_extract_dir)[:20]}"
                )

        # Auto-detect delimiter
        with open(csv_path, 'r', encoding='utf-8') as f:
            first_line = f.readline()
        csv_delimiter = '\t' if first_line.count('\t') > first_line.count(',') else ','

        print(f"   Using metadata: {os.path.basename(csv_path)} "
              f"(delimiter: {'TAB' if csv_delimiter == chr(9) else 'COMMA'})")

        # ── Step 4: Find the audio directory ──────────────────────
        # Expected: audio_files/ (this dataset), audios/, or clips/
        audios_dir = None
        for candidate_name in ["audio_files", "audios", "clips", "audio"]:
            candidate_path = os.path.join(cv_extract_dir, candidate_name)
            if os.path.isdir(candidate_path):
                sample_files = os.listdir(candidate_path)[:5]
                if any(f.endswith(('.mp3', '.wav', '.opus', '.flac')) for f in sample_files):
                    audios_dir = candidate_path
                    break

        if audios_dir is None:
            # Maybe audio files are directly in cv_extract_dir
            if any(f.endswith('.mp3') for f in os.listdir(cv_extract_dir)[:100]):
                audios_dir = cv_extract_dir
            else:
                raise FileNotFoundError(
                    f"No audio directory found in {cv_extract_dir}. "
                    f"Subdirs: {[d for d in os.listdir(cv_extract_dir) if os.path.isdir(os.path.join(cv_extract_dir, d))]}"
                )

        num_audio_files = len([f for f in os.listdir(audios_dir)
                               if f.endswith(('.mp3', '.wav'))])
        print(f"   📂 Audio directory: {audios_dir} ({num_audio_files} files)")

        # ── Step 5: Load samples from CSV ─────────────────────────
        # This is an en-AU SUBSET — ALL rows are Australian accented,
        # so we do NOT filter by accent. Just load everything.

        max_cv_samples = int(CV_ACCENT_HOURS_CAP * 600)  # ~40h cap

        row_count = 0
        skipped_no_audio = 0
        skipped_duration = 0
        skipped_no_text = 0
        accent_counter = Counter()

        print(f"   Loading all Australian accent samples (cap: {max_cv_samples})...")

        # ── DEBUG: Show what paths look like vs actual files ───────
        sample_audio_files = sorted(os.listdir(audios_dir))[:5]
        print(f"   Sample files in audio_files/: {sample_audio_files}")

        with open(csv_path, "r", encoding="utf-8") as f:
            debug_reader = csv.DictReader(f, delimiter=csv_delimiter)
            debug_paths = []
            for i, row in enumerate(debug_reader):
                if i >= 5:
                    break
                debug_paths.append(row.get("path") or row.get("filename") or "")
            print(f"   Sample 'path' values in CSV: {debug_paths}")

        # Build an index of actual audio files for flexible matching
        # This handles renamed files, missing extensions, etc.
        print(f"   Building audio file index...")
        audio_file_index = {}   # basename (no ext) → full path
        audio_file_exact = {}   # exact filename → full path
        for af in os.listdir(audios_dir):
            af_path = os.path.join(audios_dir, af)
            if os.path.isfile(af_path):
                audio_file_exact[af] = af_path
                stem = os.path.splitext(af)[0]
                audio_file_index[stem] = af_path
        print(f"   Indexed {len(audio_file_exact)} audio files")

        with open(csv_path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f, delimiter=csv_delimiter)
            print(f"   CSV columns: {reader.fieldnames}")

            for row in reader:
                row_count += 1

                if row_count % 10000 == 0:
                    print(f"   ... processed {row_count} rows, "
                          f"loaded {len(cv_samples)} samples "
                          f"(skip: text={skipped_no_text}, audio={skipped_no_audio}, dur={skipped_duration})")

                # ── Get sentence text ──
                sentence = (
                    row.get("sentence")
                    or row.get("text")
                    or row.get("transcription")
                    or ""
                )
                if not sentence or len(sentence.strip()) < 2:
                    skipped_no_text += 1
                    continue

                # ── Track accent distribution ──
                accent = (
                    row.get("accents")
                    or row.get("accent")
                    or "australian"
                ).strip()
                accent_counter[accent] += 1

                # ── Resolve audio file path ──
                clip_filename = row.get("path") or row.get("filename") or ""
                if not clip_filename:
                    skipped_no_audio += 1
                    continue

                # Strip any directory prefix (e.g., "en/clips/file.mp3" → "file.mp3")
                clip_basename = os.path.basename(clip_filename)

                # Try with and without .mp3 extension
                candidates_to_try = [clip_basename, clip_filename]
                if not clip_basename.endswith((".mp3", ".wav", ".opus", ".flac")):
                    candidates_to_try.insert(0, clip_basename + ".mp3")
                    candidates_to_try.append(clip_filename + ".mp3")

                clip_path = None
                search_dirs = [audios_dir, cv_extract_dir,
                               os.path.join(cv_extract_dir, "clips"),
                               os.path.join(cv_extract_dir, "audios"),
                               os.path.join(cv_extract_dir, "audio_files")]
                for try_dir in search_dirs:
                    if not os.path.isdir(try_dir):
                        continue
                    for candidate_name in candidates_to_try:
                        candidate = os.path.join(try_dir, candidate_name)
                        if os.path.exists(candidate):
                            clip_path = candidate
                            break
                    if clip_path:
                        break

                # Fallback: use the audio file index (flexible matching)
                if clip_path is None:
                    # Try exact match on basename
                    if clip_basename in audio_file_exact:
                        clip_path = audio_file_exact[clip_basename]
                    else:
                        # Try stem match (without extension)
                        stem = os.path.splitext(clip_basename)[0]
                        if stem in audio_file_index:
                            clip_path = audio_file_index[stem]

                if clip_path is None:
                    skipped_no_audio += 1
                    if skipped_no_audio <= 3:
                        print(f"   DEBUG: No audio found for path='{clip_filename}'")
                        print(f"   DEBUG: Tried: {os.path.join(audios_dir, clip_filename)}")
                    continue

                # ── Duration check (use metadata first, saves I/O) ──
                duration_ms = row.get("duration_ms")
                if duration_ms is not None:
                    try:
                        raw_dur = float(duration_ms)
                        # The column is called "duration_ms" but in this dataset
                        # values are actually in SECONDS (e.g., 7.39, not 7390).
                        # Heuristic: if value < 200, it's seconds; otherwise ms.
                        duration_s = raw_dur if raw_dur < 200 else raw_dur / 1000.0
                        if duration_s < 0.5 or duration_s > MAX_AUDIO_LENGTH_S:
                            skipped_duration += 1
                            continue
                    except (ValueError, TypeError):
                        pass

                # ── Load audio with librosa (already returns float32) ──
                try:
                    audio_array, sr = librosa.load(
                        clip_path, sr=SAMPLING_RATE, mono=True
                    )
                    # FIX: Ensure float32 (librosa usually returns float32,
                    # but we enforce it explicitly for consistency)
                    audio_array = audio_array.astype(np.float32)
                except Exception:
                    skipped_no_audio += 1
                    continue

                duration = len(audio_array) / sr
                if duration < 0.5 or duration > MAX_AUDIO_LENGTH_S:
                    skipped_duration += 1
                    continue

                cv_samples.append({
                    "audio": {
                        "array": audio_array,
                        "sampling_rate": SAMPLING_RATE,
                    },
                    "text": sentence.strip().upper(),
                    "source": "common_voice_au",
                })

                if len(cv_samples) >= max_cv_samples:
                    print(f"   Reached cap of {max_cv_samples} samples.")
                    break

        print(f"\n   ✅ Loaded {len(cv_samples)} Australian accent samples "
              f"from Common Voice v24 (MDC)")
        print(f"      Total rows in CSV:  {row_count}")
        print(f"      Skipped (no text):  {skipped_no_text}")
        print(f"      Skipped (no audio): {skipped_no_audio}")
        print(f"      Skipped (duration): {skipped_duration}")
        print(f"   Accent distribution:")
        for acc, cnt in accent_counter.most_common():
            print(f"      {acc}: {cnt}")

        cv_loaded = True

    except ImportError as e:
        print(f"   ⚠️  datacollective SDK import failed: {e}")
        print("   Make sure you ran: pip install datacollective")
    except Exception as e:
        print(f"   ⚠️  MDC download/processing failed: {e}")
        import traceback
        traceback.print_exc()

    # ── Fallback chain (if MDC fails) ─────────────────────────────
    if not cv_loaded:
        print("\n   MDC failed. Trying fallback datasets...")

        # Fallback 1: VoxPopuli English (open, EU Parliament, accented English)
        print("   Trying VoxPopuli English (open dataset)...")
        try:
            vp = load_dataset("facebook/voxpopuli", "en", split="train",
                              streaming=True, trust_remote_code=True)
            vp_count = 0
            max_vp = 15000
            for sample in vp:
                text = sample.get("raw_text", "") or sample.get("normalized_text", "")
                if text and len(text.strip()) >= 2:
                    # FIX: normalize audio to float32
                    cv_samples.append({
                        "audio": normalize_audio_to_float32(sample["audio"]),
                        "text": text.strip().upper(),
                        "source": "voxpopuli"
                    })
                    vp_count += 1
                    if vp_count % 5000 == 0:
                        print(f"   ... VoxPopuli: {vp_count} samples loaded")
                if vp_count >= max_vp:
                    break
            print(f"   VoxPopuli English: {vp_count} samples")
            if cv_samples:
                cv_loaded = True
        except Exception as e:
            print(f"   ⚠️  VoxPopuli failed: {str(e)[:120]}")

        # Fallback 2: Google FLEURS
        if not cv_loaded:
            print("   Trying Google FLEURS...")
            try:
                fleurs = load_dataset("google/fleurs", "en_us", split="train",
                                      streaming=True, trust_remote_code=True)
                fleurs_count = 0
                for sample in fleurs:
                    text = sample.get("transcription", "")
                    if text and len(text.strip()) >= 2:
                        # FIX: normalize audio to float32
                        cv_samples.append({
                            "audio": normalize_audio_to_float32(sample["audio"]),
                            "text": text.strip().upper(),
                            "source": "fleurs"
                        })
                        fleurs_count += 1
                print(f"   FLEURS en_us: {fleurs_count} samples")

                try:
                    fleurs_gb = load_dataset("google/fleurs", "en_gb", split="train",
                                             streaming=True, trust_remote_code=True)
                    gb_count = 0
                    for sample in fleurs_gb:
                        text = sample.get("transcription", "")
                        if text and len(text.strip()) >= 2:
                            # FIX: normalize audio to float32
                            cv_samples.append({
                                "audio": normalize_audio_to_float32(sample["audio"]),
                                "text": text.strip().upper(),
                                "source": "fleurs"
                            })
                            gb_count += 1
                    print(f"   FLEURS en_gb: {gb_count} samples")
                except Exception:
                    pass

                if cv_samples:
                    cv_loaded = True
            except Exception as e:
                print(f"   ⚠️  FLEURS also failed: {str(e)[:120]}")

    # ── Add CV samples to train/eval split ────────────────────────
    if cv_samples:
        n_eval = max(1, len(cv_samples) // 20)
        all_eval_samples.extend(cv_samples[:n_eval])
        all_train_samples.extend(cv_samples[n_eval:])
        src_name = {
            "common_voice_au": "Common Voice v24 en-AU (MDC)",
            "common_voice": "Common Voice (MDC)",
            "voxpopuli": "VoxPopuli",
            "fleurs": "FLEURS"
        }.get(cv_samples[0]["source"], cv_samples[0]["source"])
        print(f"\n   {src_name}: {len(cv_samples) - n_eval} train, {n_eval} eval")
    else:
        print("   ⚠️  No supplementary accent data loaded — continuing with EdAcc + Dialects only")


# ══════════════════════════════════════════════════════════════════
# FIX: Final safety net — standardize ALL audio dicts before Arrow
#
# This does TWO things:
#   1. Ensures all arrays are float32 (dtype fix)
#   2. Ensures all audio dicts have EXACTLY the same keys:
#      {"array": np.float32, "sampling_rate": int}
#      This strips "path", "bytes", etc. from HF dataset audio dicts
#      that would cause ArrowInvalid when mixed with manually-built dicts
# ══════════════════════════════════════════════════════════════════
print("\n🔧 Standardizing all audio dicts (dtype + key consistency fix)...")
dtype_fixes = 0
key_fixes = 0
REQUIRED_AUDIO_KEYS = {"array", "sampling_rate"}

for samples_list in [all_train_samples, all_eval_samples]:
    for s in samples_list:
        audio = s.get("audio")
        if isinstance(audio, dict):
            arr = audio.get("array")
            sr = audio.get("sampling_rate", SAMPLING_RATE)

            # Fix dtype
            if arr is not None and hasattr(arr, "dtype") and arr.dtype != np.float32:
                arr = arr.astype(np.float32)
                dtype_fixes += 1

            # Fix key structure — rebuild with ONLY required keys
            current_keys = set(audio.keys())
            if current_keys != REQUIRED_AUDIO_KEYS:
                key_fixes += 1

            s["audio"] = {"array": arr, "sampling_rate": int(sr)}

print(f"   Fixed {dtype_fixes} arrays from non-float32 dtypes")
print(f"   Fixed {key_fixes} audio dicts with extra/missing keys")

# ── Debug: verify consistency before building Arrow table ─────────
print("\n🔍 Pre-flight check...")
sample_train = all_train_samples[0] if all_train_samples else None
sample_cv = None
for s in all_train_samples:
    if s.get("source") == "common_voice_au":
        sample_cv = s
        break

if sample_train:
    a = sample_train["audio"]
    print(f"   First train sample audio keys: {sorted(a.keys()) if isinstance(a, dict) else type(a)}")
    if isinstance(a, dict) and a.get("array") is not None:
        print(f"   First train sample dtype: {a['array'].dtype}, shape: {a['array'].shape}")
if sample_cv:
    a = sample_cv["audio"]
    print(f"   First CV sample audio keys:    {sorted(a.keys()) if isinstance(a, dict) else type(a)}")
    if isinstance(a, dict) and a.get("array") is not None:
        print(f"   First CV sample dtype:    {a['array'].dtype}, shape: {a['array'].shape}")

print(f"   Total train samples: {len(all_train_samples)}")
print(f"   Total eval samples:  {len(all_eval_samples)}")


# ══════════════════════════════════════════════════════════════════
# Combine into HF Datasets — STREAMING via from_generator()
#
# Dataset.from_list() and even chunked approaches crash the kernel
# because they require all data in RAM simultaneously for Arrow
# table construction. With ~46K samples of variable-length audio
# arrays, this causes silent OOM kills.
#
# Fix: Use Dataset.from_generator() which streams one sample at a
# time directly into Arrow files on disk. Peak memory = 1 sample.
# ══════════════════════════════════════════════════════════════════
print("\n📊 Building combined datasets (streaming to Arrow on disk)...")
import gc
import sys

def make_generator(samples_list, label="dataset"):
    """Return a generator function (not a generator) for Dataset.from_generator()."""
    total = len(samples_list)

    def gen():
        for i, s in enumerate(samples_list):
            if (i + 1) % 5000 == 0:
                print(f"   ... {label}: {i + 1}/{total} samples written")
                sys.stdout.flush()
            yield {
                "audio": s["audio"],
                "text":  s["text"],
                "source": s["source"],
            }

    return gen

print(f"   Writing train ({len(all_train_samples)} samples) to Arrow...")
sys.stdout.flush()
train_ds = Dataset.from_generator(
    make_generator(all_train_samples, "train"),
    cache_dir="./hf_cache_train",
)
print(f"   ✅ Train dataset: {len(train_ds)} rows")

# Free train samples before building eval
del all_train_samples
gc.collect()

print(f"   Writing eval ({len(all_eval_samples)} samples) to Arrow...")
sys.stdout.flush()
eval_ds = Dataset.from_generator(
    make_generator(all_eval_samples, "eval"),
    cache_dir="./hf_cache_eval",
)
print(f"   ✅ Eval dataset: {len(eval_ds)} rows")

del all_eval_samples
gc.collect()

# NOTE: We skip cast_column("audio", Audio(...)) for train_ds/eval_ds.
# Our audio data is already loaded as {"array": float32, "sampling_rate": 16000}
# dicts in memory. The Audio feature type is only needed for lazy decoding
# from file paths, which doesn't apply here. The prepare_dataset function
# in Cell 7 uses get_audio_array_and_sr() which handles raw dicts directly.
#
# Calling cast_column after concatenate_datasets causes a PyArrow
# ChunkedArray error due to a compatibility issue between the datasets
# library and PyArrow when casting chunked struct columns.

# Print source distribution
train_sources = Counter(train_ds["source"])
print(f"\n✅ Combined training set: {len(train_ds)} samples")
for src, count in train_sources.most_common():
    print(f"   {src}: {count} samples")
print(f"✅ Combined eval set: {len(eval_ds)} samples")

# Estimate total hours
total_samples = len(train_ds)
est_hours = total_samples * 5 / 3600  # assume ~5s average
print(f"   Estimated training hours: ~{est_hours:.0f}h")


# %% — Cell 6: Prepare EdAcc Official Test Set
# ──────────────────────────────────────────────────────────────────

print("📋 Preparing EdAcc official test set for leaderboard evaluation...")

edacc_test_ds = edacc_test_filtered.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE))
print(f"   EdAcc test set: {len(edacc_test_ds)} utterances")

if "accent" in edacc_test_ds.column_names:
    test_accents = Counter(edacc_test_ds["accent"])
    print("   Test accent distribution:")
    for accent, count in test_accents.most_common(10):
        print(f"     {accent}: {count}")


# %% — Cell 7: Encode Data for Whisper
# ──────────────────────────────────────────────────────────────────

import re

def normalize_text(text):
    """Normalize text for Whisper training."""
    text = text.upper()
    text = re.sub(r"[^\w\s']", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def prepare_dataset(batch):
    """Encode a single sample for Whisper training."""
    audio = batch["audio"]

    arr, sr = get_audio_array_and_sr(audio)
    if arr is None:
        batch["input_features"] = feature_extractor(
            [0.0] * SAMPLING_RATE, sampling_rate=SAMPLING_RATE, return_tensors="np"
        ).input_features[0]
        batch["labels"] = []
        batch["labels_length"] = 0
        return batch

    # Resample if needed (EdAcc is 32kHz, CV is already 16kHz)
    if sr != SAMPLING_RATE:
        arr = librosa.resample(arr, orig_sr=sr, target_sr=SAMPLING_RATE)
        sr = SAMPLING_RATE

    batch["input_features"] = feature_extractor(
        arr,
        sampling_rate=sr,
        return_tensors="np"
    ).input_features[0]

    text = normalize_text(batch["text"])
    batch["labels"] = tokenizer(text).input_ids
    batch["labels_length"] = len(batch["labels"])

    return batch


print("⚙️  Encoding training data...")
train_encoded = train_ds.map(
    prepare_dataset,
    remove_columns=train_ds.column_names,
    num_proc=1,
)
train_encoded = train_encoded.filter(lambda x: x["labels_length"] <= MAX_LABEL_LENGTH)
train_encoded = train_encoded.remove_columns(["labels_length"])

print("⚙️  Encoding eval data...")
eval_encoded = eval_ds.map(
    prepare_dataset,
    remove_columns=eval_ds.column_names,
    num_proc=1,
)
eval_encoded = eval_encoded.filter(lambda x: x["labels_length"] <= MAX_LABEL_LENGTH)
eval_encoded = eval_encoded.remove_columns(["labels_length"])

print(f"✅ Encoded — Train: {len(train_encoded)}, Eval: {len(eval_encoded)}")


# %% — Cell 8: Data Collator
# ──────────────────────────────────────────────────────────────────

import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": f["labels"]} for f in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

print("✅ Data collator ready.")


# %% — Cell 9: Load Model with LoRA + 4-bit Quantization
# ──────────────────────────────────────────────────────────────────

from transformers import WhisperForConditionalGeneration, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

print("🏋️  Loading Whisper large-v3...")

if LOAD_IN_4BIT and USE_LORA:
    print("   Using 4-bit quantization (QLoRA)...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    model = WhisperForConditionalGeneration.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
    )
    model = prepare_model_for_kbit_training(model)
else:
    model = WhisperForConditionalGeneration.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        device_map="auto",
    )

if USE_LORA:
    print(f"   Applying LoRA (r={LORA_R}, alpha={LORA_ALPHA})...")
    lora_config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=["q_proj", "v_proj", "k_proj", "out_proj", "fc1", "fc2"],
        lora_dropout=0.05,
        bias="none",
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []
model.config.use_cache = False
model.generation_config.language = "english"
model.generation_config.task = "transcribe"

# Remove generation params from model.config to avoid ValueError
# "You have modified the pretrained model configuration to control generation"
for _key in ["suppress_tokens", "forced_decoder_ids", "begin_suppress_tokens"]:
    if hasattr(model.config, _key):
        delattr(model.config, _key)

model.gradient_checkpointing_enable()

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n📊 Total params:     {total_params / 1e6:.1f}M")
print(f"   Trainable params: {trainable_params / 1e6:.1f}M ({100 * trainable_params / total_params:.2f}%)")
print(f"✅ Model loaded and ready for training.")


# %% — Cell 10: Metrics
# ──────────────────────────────────────────────────────────────────

import evaluate

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    pred_str = [normalize_text(p) for p in pred_str]
    label_str = [normalize_text(l) for l in label_str]

    pairs = [(p, l) for p, l in zip(pred_str, label_str) if l.strip()]
    if not pairs:
        return {"wer": 1.0}
    pred_str, label_str = zip(*pairs)

    wer = wer_metric.compute(predictions=list(pred_str), references=list(label_str))
    return {"wer": wer}

print("✅ WER metric ready.")


# %% — Cell 10b: Target Accent Evaluation Helper + Baseline
# ──────────────────────────────────────────────────────────────────
# Define a reusable function that evaluates on our target accents
# (Australian, British Isles, Irish). Run it BEFORE training to get
# baseline numbers, then again AFTER training for comparison.
# ──────────────────────────────────────────────────────────────────

from jiwer import wer as jiwer_wer

def run_inference_on_samples(samples_iter, label="eval", max_samples=None):
    """Run model inference on an iterable of samples with 'audio' and 'text' keys."""
    preds = []
    refs = []
    count = 0
    for sample in samples_iter:
        ref_text = normalize_text(sample.get("text", ""))
        if not ref_text.strip():
            continue

        audio = sample.get("audio") or sample
        arr, sr = get_audio_array_and_sr(audio)
        if arr is None:
            continue

        if sr != SAMPLING_RATE:
            arr = librosa.resample(arr, orig_sr=sr, target_sr=SAMPLING_RATE)
            sr = SAMPLING_RATE

        input_features = feature_extractor(
            arr, sampling_rate=sr, return_tensors="pt"
        ).input_features.to(model.device)

        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                max_new_tokens=225,
                num_beams=5,
                language="english",
                task="transcribe",
            )

        pred_text = normalize_text(
            tokenizer.decode(predicted_ids[0], skip_special_tokens=True)
        )
        preds.append(pred_text)
        refs.append(ref_text)
        count += 1

        if count % 200 == 0:
            print(f"      ... {label}: {count} samples processed")

        if max_samples and count >= max_samples:
            break

    return preds, refs


def evaluate_target_accents(phase="base"):
    """Evaluate model on Australian, British Isles, and Irish English.
    Returns dict of WERs for comparison."""
    print(f"\n{'=' * 70}")
    print(f"TARGET ACCENT EVALUATION ({phase.upper()})")
    print(f"{'=' * 70}")

    model.eval()
    results = {}

    # ── 1. Australian English ──
    print(f"\n   🇦🇺 Australian English (Common Voice en-AU eval)...")
    au_indices = [i for i, s in enumerate(eval_ds["source"]) if s == "common_voice_au"]
    if au_indices:
        au_eval = eval_ds.select(au_indices)
        au_preds, au_refs = run_inference_on_samples(au_eval, label="AU")
        results["au_wer"] = jiwer_wer(au_refs, au_preds)
        results["au_n"] = len(au_refs)
        print(f"      Australian WER: {results['au_wer']*100:.2f}% ({results['au_n']} samples)")
    else:
        print(f"      ⚠️  No Australian eval samples found")

    # ── 2. British Isles dialects (per-region) ──
    print(f"\n   🇬🇧 British Isles dialects...")
    dialect_results_local = {}
    all_dialect_preds = []
    all_dialect_refs = []

    for config_name in DIALECT_CONFIGS:
        try:
            ds = load_dataset("ylacombe/english_dialects", config_name,
                              split="train", streaming=True)
            all_config_samples = []
            for sample in ds:
                text = sample.get("text", "")
                if text and len(text.strip()) >= 2:
                    all_config_samples.append(sample)

            if not all_config_samples:
                continue

            n_eval = max(5, len(all_config_samples) // 20)
            eval_samples = all_config_samples[:n_eval]

            preds, refs_local = run_inference_on_samples(eval_samples, label=config_name)

            if refs_local:
                region = config_name.rsplit('_', 1)[0]
                if region not in dialect_results_local:
                    dialect_results_local[region] = {"preds": [], "refs": []}
                dialect_results_local[region]["preds"].extend(preds)
                dialect_results_local[region]["refs"].extend(refs_local)
                all_dialect_preds.extend(preds)
                all_dialect_refs.extend(refs_local)

        except Exception as e:
            print(f"      ⚠️  {config_name}: {str(e)[:60]}")

    if all_dialect_refs:
        results["dialect_wer"] = jiwer_wer(all_dialect_refs, all_dialect_preds)
        results["dialect_n"] = len(all_dialect_refs)
        print(f"      British Isles overall WER: {results['dialect_wer']*100:.2f}% ({results['dialect_n']} samples)")

        results["per_region"] = {}
        for region, data in sorted(dialect_results_local.items()):
            if data["refs"]:
                rwer = jiwer_wer(data["refs"], data["preds"])
                results["per_region"][region] = {"wer": rwer, "n": len(data["refs"])}
                print(f"         {region}: {rwer*100:.1f}% ({len(data['refs'])} samples)")

    # ── 3. Irish English from EdAcc ──
    print(f"\n   🇮🇪 Irish English (EdAcc test)...")
    if "accent" in edacc_test_ds.column_names:
        irish_indices = [i for i, a in enumerate(edacc_test_ds["accent"]) if "Irish" in str(a)]
        if irish_indices:
            irish_eval = edacc_test_ds.select(irish_indices)
            irish_preds, irish_refs = run_inference_on_samples(irish_eval, label="Irish-EdAcc")
            results["irish_wer"] = jiwer_wer(irish_refs, irish_preds)
            results["irish_n"] = len(irish_refs)
            print(f"      Irish English WER: {results['irish_wer']*100:.2f}% ({results['irish_n']} samples)")

    return results


# ── Run BASELINE evaluation before training ──
print("\n📊 Running BASELINE target accent evaluation (before fine-tuning)...")
print("   This takes ~15-20 minutes but gives you comparison numbers.")
base_target_results = evaluate_target_accents(phase="baseline (pre-fine-tuning)")


# %% — Cell 11: Training
# ──────────────────────────────────────────────────────────────────

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-large-v3-accented-ft",

    # Batch & memory
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    gradient_checkpointing=True,
    bf16=True,    
    fp16=False,
    dataloader_num_workers=NUM_WORKERS,
    dataloader_pin_memory=True,

    # Optimization
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    weight_decay=0.01,
    max_grad_norm=1.0,
    optim="adamw_torch",
    lr_scheduler_type="cosine",

    # Eval & save
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    save_strategy="steps",
    save_steps=EVAL_STEPS,
    save_total_limit=SAVE_TOTAL_LIMIT,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,

    # Generation (for eval)
    predict_with_generate=True,
    generation_max_length=225,

    # Logging
    logging_steps=25,
    logging_first_step=True,
    report_to="none",

    # Remove unused columns
    remove_unused_columns=False,
)

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=eval_encoded,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor.feature_extractor,
)

# ── Evaluate base model ──
print("📊 Evaluating BASE Whisper large-v3 (before fine-tuning)...")
base_metrics = trainer.evaluate()
base_wer = base_metrics["eval_wer"]
print(f"   Base WER: {base_wer:.4f} ({base_wer*100:.2f}%)")

# ── Train ──
total_steps = (len(train_encoded) // (BATCH_SIZE * GRAD_ACCUM)) * NUM_EPOCHS
print(f"\n🚂 Training for {NUM_EPOCHS} epochs (~{total_steps} steps)...")
print(f"   This will take a while with {len(train_encoded)} samples...")
trainer.train()

# ── Final eval on training eval set ──
ft_metrics = trainer.evaluate()
ft_wer = ft_metrics["eval_wer"]
print(f"\n📊 Training Results:")
print(f"   Base WER:  {base_wer:.4f} ({base_wer*100:.2f}%)")
print(f"   FT WER:    {ft_wer:.4f} ({ft_wer*100:.2f}%)")
print(f"   Improvement: {(base_wer - ft_wer) / base_wer * 100:.1f}%")


# %% — Cell 12: Save Model
# ──────────────────────────────────────────────────────────────────

SAVE_DIR = "./whisper-large-v3-accent-sota"

print(f"💾 Saving fine-tuned model to {SAVE_DIR}...")
if USE_LORA:
    model.save_pretrained(SAVE_DIR)
    processor.save_pretrained(SAVE_DIR)
    print(f"   Saved LoRA adapters + processor to {SAVE_DIR}")
else:
    model.save_pretrained(SAVE_DIR)
    processor.save_pretrained(SAVE_DIR)
    print(f"   Saved full model to {SAVE_DIR}")


# %% — Cell 13: Official EdAcc Leaderboard Evaluation
# ──────────────────────────────────────────────────────────────────

from jiwer import wer as jiwer_wer
import numpy as np
from tqdm import tqdm

print("=" * 70)
print("OFFICIAL EdAcc TEST SET EVALUATION")
print("=" * 70)
print(f"Evaluating on {len(edacc_test_ds)} test utterances...")

model.eval()

all_preds = []
all_refs = []
accent_results = {}

EVAL_BATCH = 8

for i in tqdm(range(0, len(edacc_test_ds), EVAL_BATCH)):
    batch_samples = edacc_test_ds.select(range(i, min(i + EVAL_BATCH, len(edacc_test_ds))))

    for sample in batch_samples:
        audio = sample["audio"]
        ref_text = normalize_text(sample["text"])

        if not ref_text.strip():
            continue

        arr, sr = get_audio_array_and_sr(audio)
        if arr is None:
            continue

        # Resample if needed (EdAcc test is 32kHz)
        if sr != SAMPLING_RATE:
            arr = librosa.resample(arr, orig_sr=sr, target_sr=SAMPLING_RATE)
            sr = SAMPLING_RATE

        input_features = feature_extractor(
            arr,
            sampling_rate=sr,
            return_tensors="pt"
        ).input_features.to(model.device)

        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                max_new_tokens=225,
                num_beams=5,
                language="english",
                task="transcribe",
            )

        pred_text = normalize_text(
            tokenizer.decode(predicted_ids[0], skip_special_tokens=True)
        )

        all_preds.append(pred_text)
        all_refs.append(ref_text)

        accent = sample.get("accent", "unknown")
        if accent not in accent_results:
            accent_results[accent] = {"preds": [], "refs": []}
        accent_results[accent]["preds"].append(pred_text)
        accent_results[accent]["refs"].append(ref_text)

# ── Compute overall WER ──
overall_wer = jiwer_wer(all_refs, all_preds)

print(f"\n{'=' * 70}")
print(f"RESULTS")
print(f"{'=' * 70}")
print(f"\n🎯 Overall EdAcc Test WER: {overall_wer:.4f} ({overall_wer*100:.2f}%)")
print(f"\n   Leaderboard comparison:")
print(f"   ┌─────────────────────────────────┬───────────┐")
print(f"   │ Model                           │ Test WER  │")
print(f"   ├─────────────────────────────────┼───────────┤")
print(f"   │ Anonymous Commercial (SOTA)     │ 18.7%     │")
print(f"   │ Whisper Large (zero-shot)       │ 19.7%     │")
print(f"   │ ★ Ours (Whisper-large-v3 + LoRA)│ {overall_wer*100:.1f}%     │")
print(f"   │ Wav2Vec 2.0 + LM               │ 36.1%     │")
print(f"   └─────────────────────────────────┴───────────┘")

if overall_wer < 0.187:
    print(f"\n🏆 NEW STATE OF THE ART! Beat the previous best of 18.7%!")
elif overall_wer < 0.197:
    print(f"\n🥈 Beat Whisper Large zero-shot (19.7%), close to SOTA!")
else:
    print(f"\n📈 Room for improvement — try more epochs or more data.")


# ── Per-accent breakdown ──
print(f"\n📊 Per-Accent WER Breakdown:")
print(f"   {'Accent':<30} {'WER':>8} {'Samples':>8}")
print(f"   {'─' * 48}")

accent_wers = []
for accent, data in sorted(accent_results.items(),
                            key=lambda x: jiwer_wer(x[1]["refs"], x[1]["preds"])):
    if len(data["refs"]) >= 5:
        accent_wer = jiwer_wer(data["refs"], data["preds"])
        accent_wers.append((accent, accent_wer, len(data["refs"])))
        marker = "✅" if accent_wer < 0.20 else "⚠️" if accent_wer < 0.30 else "❌"
        print(f"   {marker} {accent:<28} {accent_wer*100:>7.1f}% {len(data['refs']):>7}")

print(f"\n🔍 Hardest accents (highest WER):")
for accent, awer, n in sorted(accent_wers, key=lambda x: -x[1])[:5]:
    print(f"   {accent}: {awer*100:.1f}% ({n} samples)")


# %% — Cell 14: TARGET ACCENT BENCHMARK (British / Irish / Aussie)
# ──────────────────────────────────────────────────────────────────
#
# This is the benchmark that matters for our goal.
# Runs the SAME evaluation as the baseline (Cell 10b) on the
# fine-tuned model, then shows a side-by-side comparison.
#
# ──────────────────────────────────────────────────────────────────

print("\n📊 Running FINE-TUNED target accent evaluation...")
ft_target_results = evaluate_target_accents(phase="fine-tuned (post-training)")


# ══════════════════════════════════════════════════════════════════
# SIDE-BY-SIDE COMPARISON: BASE vs FINE-TUNED
# ══════════════════════════════════════════════════════════════════
print(f"\n{'=' * 70}")
print(f"🎯 TARGET ACCENT RESULTS: BASE vs FINE-TUNED")
print(f"{'=' * 70}")

def fmt_wer(results, key):
    """Format WER from results dict, return '  N/A  ' if missing."""
    if key in results:
        return f"{results[key]*100:>6.1f}%"
    return "   N/A "

def fmt_delta(base_results, ft_results, key):
    """Format improvement delta."""
    if key in base_results and key in ft_results:
        base_val = base_results[key]
        ft_val = ft_results[key]
        delta = (base_val - ft_val) / base_val * 100
        if delta > 0:
            return f"  ↓{delta:>4.1f}%"
        elif delta < 0:
            return f"  ↑{abs(delta):>4.1f}%"
        else:
            return "     0%"
    return "      —"

print(f"")
print(f"   ┌─────────────────────────────┬──────────┬──────────┬─────────┬─────────┐")
print(f"   │ Accent / Region             │ Base WER │  FT WER  │ Change  │ Samples │")
print(f"   ├─────────────────────────────┼──────────┼──────────┼─────────┼─────────┤")

# Australian
print(f"   │ 🇦🇺 Australian English       │ {fmt_wer(base_target_results, 'au_wer')} │ {fmt_wer(ft_target_results, 'au_wer')} │{fmt_delta(base_target_results, ft_target_results, 'au_wer')} │ {ft_target_results.get('au_n', base_target_results.get('au_n', '?')):>7} │")

# British Isles overall
print(f"   │ 🇬🇧 British Isles (all)       │ {fmt_wer(base_target_results, 'dialect_wer')} │ {fmt_wer(ft_target_results, 'dialect_wer')} │{fmt_delta(base_target_results, ft_target_results, 'dialect_wer')} │ {ft_target_results.get('dialect_n', base_target_results.get('dialect_n', '?')):>7} │")

# Per-region breakdown
all_regions = set()
if "per_region" in base_target_results:
    all_regions.update(base_target_results["per_region"].keys())
if "per_region" in ft_target_results:
    all_regions.update(ft_target_results["per_region"].keys())

for region in sorted(all_regions):
    base_r = base_target_results.get("per_region", {}).get(region, {})
    ft_r = ft_target_results.get("per_region", {}).get(region, {})
    base_w = f"{base_r['wer']*100:>6.1f}%" if "wer" in base_r else "   N/A "
    ft_w = f"{ft_r['wer']*100:>6.1f}%" if "wer" in ft_r else "   N/A "
    if "wer" in base_r and "wer" in ft_r:
        d = (base_r["wer"] - ft_r["wer"]) / base_r["wer"] * 100
        delta = f"  ↓{d:>4.1f}%" if d > 0 else f"  ↑{abs(d):>4.1f}%" if d < 0 else "     0%"
    else:
        delta = "      —"
    n = ft_r.get("n", base_r.get("n", "?"))
    print(f"   │    ├─ {region:<20} │ {base_w} │ {ft_w} │{delta} │ {n:>7} │")

# Irish English (EdAcc)
print(f"   │ 🇮🇪 Irish English (EdAcc)     │ {fmt_wer(base_target_results, 'irish_wer')} │ {fmt_wer(ft_target_results, 'irish_wer')} │{fmt_delta(base_target_results, ft_target_results, 'irish_wer')} │ {ft_target_results.get('irish_n', base_target_results.get('irish_n', '?')):>7} │")

print(f"   ├─────────────────────────────┼──────────┼──────────┼─────────┼─────────┤")

# EdAcc overall (from Cell 13)
if overall_wer is not None:
    print(f"   │ 🌍 EdAcc overall (ref)       │   —      │ {overall_wer*100:>6.1f}% │    —    │ {len(all_refs):>7} │")

print(f"   └─────────────────────────────┴──────────┴──────────┴─────────┴─────────┘")

# ── Headline improvement numbers ──
print(f"\n   📈 Improvement summary:")
for label, key in [("Australian", "au_wer"), ("British Isles", "dialect_wer"), ("Irish (EdAcc)", "irish_wer")]:
    if key in base_target_results and key in ft_target_results:
        b = base_target_results[key] * 100
        f_val = ft_target_results[key] * 100
        delta = b - f_val
        pct = delta / b * 100
        marker = "✅" if delta > 0 else "❌"
        print(f"   {marker} {label}: {b:.1f}% → {f_val:.1f}% (Δ {delta:+.1f}pp, {pct:+.1f}% relative)")


# %% — Cell 15: Example Transcriptions
# ──────────────────────────────────────────────────────────────────

import random

print("\n" + "=" * 70)
print("EXAMPLE TRANSCRIPTIONS FROM EdAcc TEST SET")
print("=" * 70)

sample_indices = random.sample(range(len(edacc_test_ds)), min(8, len(edacc_test_ds)))

for idx in sample_indices:
    sample = edacc_test_ds[idx]
    ref = normalize_text(sample["text"])
    if not ref.strip():
        continue

    accent = sample.get("accent", "unknown")

    arr, sr = get_audio_array_and_sr(sample["audio"])
    if arr is None:
        continue

    # Resample if needed
    if sr != SAMPLING_RATE:
        arr = librosa.resample(arr, orig_sr=sr, target_sr=SAMPLING_RATE)
        sr = SAMPLING_RATE

    input_features = feature_extractor(
        arr,
        sampling_rate=sr,
        return_tensors="pt"
    ).input_features.to(model.device)

    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            max_new_tokens=225,
            num_beams=5,
            language="english",
            task="transcribe",
        )

    pred = normalize_text(tokenizer.decode(predicted_ids[0], skip_special_tokens=True))
    sample_wer = jiwer_wer(ref, pred) if ref.strip() else float('inf')

    print(f"\n--- [{accent}] ---")
    print(f"  REF:  {ref}")
    print(f"  PRED: {pred}")
    print(f"  WER:  {sample_wer:.2f}")


# %% — Cell 16: Optional — Push to Hub
# ──────────────────────────────────────────────────────────────────

# Uncomment to push your model to HuggingFace Hub:
#
# HUB_REPO = "your-username/whisper-large-v3-accented-english-sota"
#
# model.push_to_hub(HUB_REPO, token=HF_TOKEN)
# processor.push_to_hub(HUB_REPO, token=HF_TOKEN)
#
# print(f"🚀 Model pushed to https://huggingface.co/{HUB_REPO}")

print("\n🎉 Done! Model saved locally. Uncomment Cell 15 to push to HuggingFace Hub.")

📦 [1/5] Core packages...


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2026.2.0 which is incompatible.


📦 [2/5] PyTorch...
📦 [3/5] HuggingFace, audio, evaluation...
📦 [4/5] LoRA + quantization (PEFT + bitsandbytes)...
📦 [5/5] Mozilla Data Collective SDK...

✅ All packages installed.
⚠️  IMPORTANT: RESTART your runtime now (Runtime → Restart runtime).
   Then skip this cell and continue from Cell 2.


/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✅ Authenticated (HF + MDC). torch=2.10.0+cu128
   CUDA: True
   GPU: NVIDIA A10
   VRAM: 23.7 GB
CONFIGURATION
  Model:       openai/whisper-large-v3
  LoRA:        r=32, alpha=64, 4-bit=True
  Epochs:      15
  Effective BS: 32
  LR:          0.0001
  Device:      cuda
  MDC Dataset: cmko7havo02f5nw07rbwwhowe
📦 Loading Whisper large-v3 processor...
✅ Processor loaded. Vocab size: 50257
LOADING ACCENT DATASETS

📥 [1/3] Loading EdAcc (40h, diverse international accents)...


...

      ... Irish-EdAcc: 200 samples processed


Both `max_new_tokens` (=225) and `max_length`(=448) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=225) and `max_length`(=448) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=225) and `max_length`(=448) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=225) and `max_length`(=448) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


      Irish English WER: 16.65% (247 samples)
📊 Evaluating BASE Whisper large-v3 (before fine-tuning)...


   Base WER: 0.1219 (12.19%)

🚂 Training for 15 epochs (~21765 steps)...
   This will take a while with 46433 samples...


Step,Training Loss,Validation Loss,Model Preparation Time,Wer
500,1.033252,0.179959,0.069900,0.134172
1000,0.826804,0.166081,0.069900,0.119841
1500,0.506715,0.160432,0.069900,0.105222
2000,0.653101,0.161552,0.069900,0.098372
2500,0.622108,0.160073,0.069900,0.112847
3000,0.422762,0.167797,0.069900,0.095448
3500,0.320375,0.169244,0.069900,0.089486
4000,0.346373,0.168451,0.069900,0.096108
